In [1]:
import os, io
import pandas as pd
import pysam
import pickle
import pybedtools
print(pybedtools.__file__)
pybedtools.helpers.set_tempdir('/data/private/pdutta_new/pybedtools_temp')
from IPython.display import display, HTML

/shared/software/anaconda/anaconda3/lib/python3.10/site-packages/pybedtools/__init__.py


In [2]:
cancer_type="Brain" 

In [3]:
vcf_folder_path="/data/projects/GDC_Cancer_Wise/Old/{}/Data/TCGA_BAM_VCF/VCF_Patientwise".format(cancer_type)
reference_genome_path="/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa"

In [4]:
non_coding_region="core_prom"

In [5]:
df= pd.read_csv("/data/private/pdutta_new/non_coding_regions/unique_{}_bed.tsv".format(non_coding_region), sep="\t")
columns = ['Chr', 'start', 'end', 'gene_id', 'transcript_id', 'exon_id', 'strand', 'TSS']
df = df[columns]
# Renaming columns
df.rename(columns={'Chr': 'chr_name'}, inplace=True)
df

,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS
0,chr1,11824,11913,['ENSG00000223972.5'],['ENST00000456328.2'],['ENSE00002234944.1'],+,11869
1,chr1,11965,12054,['ENSG00000223972.5'],['ENST00000450305.2'],['ENSE00001948541.1'],+,12010
2,chr1,29509,29598,['ENSG00000243485.5'],['ENST00000473358.1'],['ENSE00001947070.1'],+,29554
3,chr1,30222,30311,['ENSG00000243485.5'],['ENST00000469289.1'],['ENSE00001841699.1'],+,30267
4,chr1,30321,30410,['ENSG00000284332.1'],['ENST00000607096.1'],['ENSE00003695741.1'],+,30366
...,...,...,...,...,...,...,...,...
209272,chrY,57016052,57016141,['ENSG00000237801.6_PAR_Y'],['ENST00000412936.6_PAR_Y'],['ENSE00001527118.2'],-,57016096
209273,chrY,57165801,57165890,['ENSG00000228410.6_PAR_Y'],['ENST00000456370.6_PAR_Y'],['ENSE00001674980.1'],-,57165845
209274,chrY,57172725,57172814,['ENSG00000223484.7_PAR_Y'],['ENST00000421233.6_PAR_Y'],['ENSE00001611457.1'],-,57172769
209275,chrY,57203313,57203402,['ENSG00000185203.12_PAR_Y'],['ENST00000399966.9_PAR_Y'],['ENSE00001772131.3'],-,57203357


In [6]:
core_prom_bed = pybedtools.BedTool.from_dataframe(df)

In [7]:
reference_fasta = pysam.FastaFile(reference_genome_path)

In [8]:
def get_germline_vcf_files(root_folder):
    germline_files = []

    # Walk through the directory tree
    for dirpath, dirnames, filenames in os.walk(root_folder):
        # If "logs" is in dirnames, remove it to avoid traversing it
        if 'logs' in dirnames:
            dirnames.remove('logs')

        # Add only the filenames with "Germline.vcf" in the current directory to the germline_files list
        for filename in filenames:
            if "Germline.vcf" in filename:
                germline_files.append(os.path.join(dirpath, filename))
    return germline_files

In [9]:
def vcf_to_dataframe(vcf_path):
    """
    Convert a .vcf.gz file into a pandas DataFrame.

    Parameters:
    - vcf_path (str): path to the .vcf.gz file

    Returns:
    - pd.DataFrame: VCF data as a DataFrame
    """
    df = pd.read_csv(vcf_path, sep=",")
    df["START_POS"] = df["POS"]-1
    df['END_POS'] = df["START_POS"] + df['REF'].str.len()
    df = df[["CHROM", "START_POS",'END_POS', "REF", "ALT"]]
    #display(df)
    return df

In [12]:
files = get_germline_vcf_files(vcf_folder_path)

In [13]:
len(files)

154

In [17]:
intersected_vcf_data = {}
df_statistics  = pd.DataFrame(columns= ["filename", "Patient_ID",  'VCF_instance','VCF_feature','Intersected_instances', 'VCF_column_names' ])
for file_path in files:
    file_name =  file_path.split('/')[-1]
    parts = file_name.split('.')[0].split('_')
    print(parts, file_path, parts)
    df_vcf = vcf_to_dataframe(file_path)
    if df_vcf is None:
        continue
    vcf_bed = pybedtools.BedTool.from_dataframe(df_vcf)
    intersect_vcf_core_prom = core_prom_bed.intersect(vcf_bed, wa=True, wb=True)
    column_list  = df.columns.to_list() + df_vcf.columns.to_list()
    df_intersection = intersect_vcf_core_prom.to_dataframe(names=column_list)
    #print(df_intersection.shape)
    #df_intersection = df_intersection[(df_intersection['REF'].str.len() < 10) & (df_intersection['ALT'].str.len() < 10)]
    print(df_intersection.shape)
    # # print(df_intersection[['REF','ALT']])
    # # input()
    # #print(df_intersection)
    # #print(df_statistics)
    # #input()
    #gdc_tag = parts[-4]
    patient_ID = parts[0]
    #work_flow = parts[2]
    instance =  df_vcf.shape[0]
    feature = df_vcf.shape[1]
    intersected_instance = df_intersection.shape[0]
    feature_list = list(df_vcf.columns)
    print(patient_ID, instance, feature, intersected_instance,  feature_list)
    df_statistics.loc[len(df_statistics)] = [file_name, patient_ID, instance, feature,intersected_instance,  feature_list]
    intersected_vcf_data[patient_ID] = df_intersection

['TCGA-02-2483-01A-01D-1494-08', 'TCGA-02-2483-10A-01D-1494-08', 'Germline'] /data/projects/GDC_Cancer_Wise/Old/Brain/Data/TCGA_BAM_VCF/VCF_Patientwise/TCGA-02-2483/TCGA-02-2483-01A-01D-1494-08_TCGA-02-2483-10A-01D-1494-08_Germline.vcf ['TCGA-02-2483-01A-01D-1494-08', 'TCGA-02-2483-10A-01D-1494-08', 'Germline']
(25840, 13)
TCGA-02-2483-01A-01D-1494-08 4269244 5 25840 ['CHROM', 'START_POS', 'END_POS', 'REF', 'ALT']
['TCGA-CS-5396-01A-02D-1465-02', 'TCGA-CS-5396-10A-01D-1465-02', 'Germline'] /data/projects/GDC_Cancer_Wise/Old/Brain/Data/TCGA_BAM_VCF/VCF_Patientwise/TCGA-CS-5396/TCGA-CS-5396-01A-02D-1465-02_TCGA-CS-5396-10A-01D-1465-02_Germline.vcf ['TCGA-CS-5396-01A-02D-1465-02', 'TCGA-CS-5396-10A-01D-1465-02', 'Germline']
(17219, 13)
TCGA-CS-5396-01A-02D-1465-02 2733430 5 17219 ['CHROM', 'START_POS', 'END_POS', 'REF', 'ALT']
['TCGA-EZ-7264-01A-11D-2024-08', 'TCGA-EZ-7264-10A-01D-2024-08', 'Germline'] /data/projects/GDC_Cancer_Wise/Old/Brain/Data/TCGA_BAM_VCF/VCF_Patientwise/TCGA-EZ-7264

In [18]:
output_path = "/data/projects/GDC_Cancer_Wise/New_data/{}/Generated_files/Intersected_Data/Germline/non_coding_regions/{}".format(cancer_type, non_coding_region)
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [19]:
df_statistics.to_csv(output_path+"/intersected_VCF_{}_statistics.tsv".format(non_coding_region), sep="\t", index=False)

In [20]:
with open(output_path+"/intersected_vcf_{}_data.pkl".format(non_coding_region), "wb") as file:
    pickle.dump(intersected_vcf_data, file)